In [1]:
import os
import osmnx as ox
import folium
# import geopandas as gpd
import pandas as pd
import geojson
from palette import rgb2hex, hand_palette
import numpy as np
from PIL import Image


In [3]:
# Снять аннотации с OSM и наложить на карту html

In [8]:
def extract_feature(map, feature_collection, features, tags, palette, opacity=0.2):
    features = features.copy()
    features.reset_index(inplace=True)
    
    for tag in tags:
        if tag not in features or not tags[tag]:
            continue
        color = palette.get(tag, {}).get('color', [255,0,255])
        geom_types = palette.get(tag, {}).get('geom_types', [])
        positive_subtags = palette.get(tag, {}).get('positive_subtags', [])
        negative_subtags = palette.get(tag, {}).get('negative_subtags', [])
        
        fs = features[pd.notna(features[tag])]#[['geometry', tag]]
        # Фильтрация по negative_subtags (если список не пустой)
        if negative_subtags:
            fs = fs[~fs[tag].isin(negative_subtags)]
        # Фильтрация по positive_subtags (если список не пустой)
        if positive_subtags:
            fs = fs[fs[tag].isin(positive_subtags)]   
        
        if geom_types:
            fs = fs[fs['geometry'].geom_type.isin(geom_types)] 
            print(tag, geom_types, fs.shape)
            for id, row in fs.iterrows():
                add_feature(map, feature_collection, row, color, opacity, tag)
        else:
            for id, row in fs.iterrows():
                add_feature(map, feature_collection, row, color, opacity, tag)
        

def add_feature(map, feature_collection, feature, color, opacity, tag):
    hexcolor = rgb2hex(color)
    draw(map, feature['geometry'], color_line=hexcolor, opacity=opacity, info=f"{tag}:{feature[tag]}")
    # Создайте объект GeoJSON для текущей фичи и добавьте его в FeatureCollection
    geojson_feature = geojson.Feature(geometry=feature['geometry'], properties={"color": color})
    geojson_feature['properties']['tag'] = tag
    geojson_feature['properties']['subtag'] = feature[tag]
    
    if 'avg_width' in hand_palette[tag]:
        if 'lanes' in feature:
            lanes = feature['lanes']
            if pd.notna(lanes):
                geojson_feature['properties']['lanes'] = lanes
        if 'width' in feature:    
            width = feature['width']
            if pd.notna(width):
                geojson_feature['properties']['width'] = width 
        
    feature_collection['features'].append(geojson_feature)
    
def draw_polygon(map, geom, color, opacity, info):
    coords = []
    for coord in geom:
        coords.append(coord[::-1])
    pg = folium.Polygon(locations=coords, 
                    color=color, 
                    fill_color=color,
                    fill_opacity=opacity)
    popup = folium.Popup(info, parse_html=True)
    popup.add_to(pg)
    pg.add_to(map)


def draw(map, geometry, color_line, color_fill=None, opacity=0.2, info='object name'):
    """
    geometry: feature['geometry']
    color_line: ['red'; 'blue'; 'green'; etc.] 
                color for <LineString>,<Point>,<Polygon>
    color_fill: ['red'; 'blue'; 'green'; etc.] 
                fill color for <Polygon>
    """
    if not color_fill:
        color_fill = color_line

    gtype = geometry.geom_type
    if gtype == 'LineString':
        coords = []
        for lon, lat in zip(geometry.coords.xy[0], geometry.coords.xy[1]):
            coords.append((lat, lon))
        folium.PolyLine(coords, color=color_line, weight=3).add_to(map)
    elif gtype == 'Point':
        lat, lon = (geometry.y, geometry.x)
        folium.CircleMarker(location=[lat, lon], 
                            radius=6, 
                            color=color_line).add_to(map)
    elif gtype == 'Polygon':
        draw_polygon(map, 
                     geometry.exterior.coords[:], 
                     color_line, 
                     opacity, 
                     info)

    elif gtype == 'MultiPolygon':
        for polygon in geometry.geoms:
            draw_polygon(map, 
                     polygon.exterior.coords[:], 
                     color_line, 
                     opacity, 
                     info)
    else:
        print("Неизвестный тип gtype:", gtype)
        
def create_html_mask(tags, bounds, name, save_folder, palette, zoom=18, opacity=0.2, rectangle=True):
    # Соберите объекты определенных типов в заданной области с помощью osmnx
    bbox = [west, south, east, north]
    features = ox.features.features_from_bbox(bbox, tags)

    # Создайте объект FeatureCollection для хранения всех данных
    feature_collection = geojson.FeatureCollection([])

    # Создайте карту folium с базовым слоем Google Satellite
    center_lat = (north+south)/2
    center_lon = (east+west)/2
    m = folium.Map(location=[center_lat, center_lon], 
                zoom_start=zoom, 
                tiles='http://mt0.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', 
                attr='Google Satellite')

    # Добавление комментария в HTML-код
    # bounds = north, south, east, west
    html_comment = f'check area bounds (north, south, east, west): {bounds}'
    # Сохранение карты в HTML-файле с комментарием
    m.get_root().html.add_child(folium.Element(html_comment))

    # Добавьте выбранные объекты на карту
    extract_feature(m, feature_collection, features, tags, palette, opacity)
        
    # Отрисовка рамки по границам     
    if rectangle:
            folium.Rectangle(((north, west), (south, east)), color='white').add_to(m)

    # Сохраните карту в HTML-файл
    if save_folder:
        m.save(f'{save_folder}/{name}_op{opacity}.html')

        # Создайте GeoJSON-файл
        with open(f'{save_folder}/{name}.geojson', 'w') as f:
            geojson.dump(feature_collection, f)
        
    return features, m
# =====================================================================
# Задайте место
north, south, east, west =  48.92085, 48.91483, 2.29661, 2.28353 # Paris
north, south, east, west =  48.92831, 48.90414, 2.35185, 2.29657 # Paris
north, south, east, west =  48.945501, 48.943991, 2.334149, 2.330695 # Paris running road
bounds = north, south, east, west

# Задайте искомые тэги объектов
tags={
    'highway': True,
    'building': True,
    'natural': True,
    'landuse': True,
    'leisure': True,
    'shop': True,
    'water': True,
    'footway': False,
    'sport':True
}

# Уровень зума (0-21)
zoom=18

# Отрисовка рамки по искомой области
rectangle = True

# Сохранение
save_folder = "data/Paris_running_road"
name = 'Paris_running_road'

# Палитра цветов
palette = hand_palette


features, map = create_html_mask(tags, bounds, name, save_folder, palette, zoom, opacity=0.2, rectangle=True)
# ======================================================================


highway ['LineString'] (4, 36)
building ['Polygon'] (10, 36)
natural ['Polygon', 'MultiPolygon'] (1, 36)
landuse ['Polygon', 'MultiPolygon'] (3, 36)
leisure ['Polygon', 'MultiPolygon'] (1, 36)
water ['Polygon', 'MultiPolygon'] (1, 36)
sport ['Polygon', 'MultiPolygon'] (1, 36)


In [ ]:
'\n'.join(map(str, [1,2,3])

TypeError: 'Map' object is not callable

In [19]:
# features[features['highway'] == 'primary']['lanes']

# mps = features[features['geometry'].geom_type == "LineString"]#[~pd.notna(features['highway'])].drop(columns=['highway', 'distance', 'nat_ref', 'operator', 'ref:highway',	'source',	'ref',	'addr:postcode',	'address',	'amenity',	'brand'])
# mps[mps['construction']=='subway']
# pd.DataFrame().drop()
# mps = mps[pd.notna(mps)]
# mps['geometry']
p = features[pd.notna(features['sport'])][features['geometry'].geom_type == 'Polygon'].iloc[0]['geometry']

print(type(p))

<class 'shapely.geometry.polygon.Polygon'>


/home/axuji/anaconda3/envs/osm/lib/python3.12/site-packages/geopandas/geodataframe.py:1750: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


In [20]:
import cv2
polygon = p

# Размер изображения
img_size = (500, 500)
image = np.random.randint(0, 256, (img_size[0], img_size[1], 3), dtype=np.uint8)

# Получаем границы (BBox)
min_x, min_y, max_x, max_y = polygon.bounds

# Функция преобразования координат в пиксели
def wgs84_to_pixels(coords, img_size, bbox):
    min_x, min_y, max_x, max_y = bbox
    scale_x = img_size[1] / (max_x - min_x)
    scale_y = img_size[0] / (max_y - min_y)
    return np.array([[
        int((x - min_x) * scale_x),
        int((max_y - y) * scale_y)  # Инверсия Y для OpenCV
    ] for x, y in coords], dtype=np.int32)

# Конвертируем в пиксельные координаты
polygon_pixel = wgs84_to_pixels(list(polygon.exterior.coords), img_size, (min_x, min_y, max_x, max_y))
holes_pixel = [wgs84_to_pixels(list(hole.coords), img_size, (min_x, min_y, max_x, max_y)) for hole in polygon.interiors]

# Создаём маску
mask = np.zeros(img_size[:2], dtype=np.uint8)
cv2.fillPoly(mask, [polygon_pixel], 255)  # Заполняем основной полигон
for hole in holes_pixel:
    cv2.fillPoly(mask, [hole], 0)  # Вырезаем дырки

# Закрашиваем область (например, зелёным)
color_layer = np.full_like(image, (0, 255, 0), dtype=np.uint8)
image[mask == 255] = color_layer[mask == 255]

# Рисуем контуры
cv2.polylines(image, [polygon_pixel], isClosed=True, color=(0, 0, 255), thickness=2)  # Внешний контур (красный)
for hole in holes_pixel:
    cv2.polylines(image, [hole], isClosed=True, color=(255, 0, 0), thickness=2)  # Внутренние контуры (синие)

# Показываем результат
cv2.imshow('Polygon from Shapely', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/axuji/anaconda3/envs/osm/lib/python3.12/site-packages/cv2/qt/plugins"


In [6]:
"""
    highway : residential, footway, | service, primary, secondary, motorway, motorway_link
    crossing # Пешеходный переход
    # crossing:markings
    # crossing:island
    bicycle
    cycleway
    oneway
    oneway:bicycle
    sidewalk
    landuse
    junction
    building
    roof:levels
    roof:shape
    source:building
    footway
"""

'\n    highway : residential, footway, | service, primary, secondary, motorway, motorway_link\n    crossing # Пешеходный переход\n    # crossing:markings\n    # crossing:island\n    bicycle\n    cycleway\n    oneway\n    oneway:bicycle\n    sidewalk\n    landuse\n    junction\n    building\n    roof:levels\n    roof:shape\n    source:building\n    footway\n'

In [2]:
# Наложить аннотации на PNG

from downloader import main as download_map#, wgs_to_tile
from simple_annotation_map import geocoordinates_to_pixels, wgs_to_tile, tile_to_lat_lon
import cv2
from tqdm import tqdm
import json
from palette import hand_palette, scale

In [5]:
# Сортировка типов для правильного порядка отрисовки:
def custom_sort_key(item):
    geometry_type = item['geometry']['type']
    tag = item['properties']['tag']

    tag_order = {'landuse': 0, 'water': 1, 
                 'building': 2, 'highway': 3, 
                 'nature': 4}
    
    if geometry_type == 'Multipolygon':
        return (0, tag_order.get(tag, 5))
    elif geometry_type == 'Polygon':
        return (1, tag_order.get(tag, 5))
    else:
        return (2, tag_order.get(tag, 5))

def world2pixels(bounds, bounds_coords, bounds_pxls):
    """
        Преобразование мировых координат объекта в пиксели
    """
    n, s, e, w = bounds
    x0,y0 = geocoordinates_to_pixels([w,n], bounds_coords, bounds_pxls)
    x1,y1 = geocoordinates_to_pixels([e,s], bounds_coords, bounds_pxls)
    return x0, y0, x1, y1

def draw_filled_multipolygon(image, multipolygon_coords, bounds_coords, bounds_pxls, color):
    """
    Заполняет пространство между полигонами в мультиполигоне, сохраняя фон внутри фигур.
    """
    mask = np.zeros(image.shape[:2], dtype=np.uint8)

    for polygon_set in multipolygon_coords:
        outer_contour = np.array([geocoordinates_to_pixels(coord, bounds_coords, bounds_pxls) 
                                  for coord in polygon_set[0]], dtype=np.int32)  # Внешний контур
        inner_contours = [np.array([geocoordinates_to_pixels(coord, bounds_coords, bounds_pxls) 
                                    for coord in hole], dtype=np.int32) for hole in polygon_set[1:]]  # Внутренние контуры

        cv2.fillPoly(mask, [outer_contour], 255)  # Заполняем основной контур
        for hole in inner_contours:
            cv2.fillPoly(mask, [hole], 0)  # Вырезаем внутренние контуры

    color_bgr = color[:3]  # Берем только 3 канала (BGR)
    color_layer = np.full_like(image, color_bgr, dtype=np.uint8)
    image[mask == 255] = color_layer[mask == 255]

    return image
    
def draw_masks(mask, features, bounds_coords, bounds_pxls, zoom, opacity=1, show=False):
    if show:
        cv2.namedWindow("mask", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("mask", 1024, 640)

    mask_info = []
    tags_dict = {}

    # Группируем объекты по тегам
    for map_obj in features:
        mask_type = map_obj['geometry']['type']
        if mask_type == 'Point':
            continue

        coordinates = map_obj['geometry']['coordinates']
        
        if mask_type == 'Polygon':
            coordinates = [[coordinates[0]] + coordinates[1:]]  # Внешний контур + внутренние отверстия
        elif mask_type == 'MultiPolygon':
            coordinates = [[[poly[0]] + poly[1:]] for poly in coordinates]  # Для каждого полигона в MultiPolygon

        color = tuple(reversed(map_obj['properties']['color']))  # Преобразуем в BGR
        alpha = int(opacity * 255)

        tag = map_obj['properties']['tag']
        if tag not in tags_dict:
            tags_dict[tag] = {'polygons': [], 'colors': []}

        transformed_polygons = []
        for poly_set in coordinates:
            projected_polygons = []
            for poly in poly_set:
                if isinstance(poly[0][0], list):  # Если `poly` содержит вложенные списки, обрабатываем их отдельно
                    processed_rings = [geocoordinates_to_pixels(ring, bounds_coords, bounds_pxls) for ring in poly]
                    projected_polygons.append(processed_rings)
                else:
                    print(poly)
                    print()
                    projected_polygons.append(geocoordinates_to_pixels(poly, bounds_coords, bounds_pxls))

            transformed_polygons.append(projected_polygons)

        tags_dict[tag]['polygons'].append(transformed_polygons)
        tags_dict[tag]['colors'].append((*color, alpha))

        mask_info.append({'tag': map_obj['properties']['tag'], 'coords': coordinates, 'color': color})

    mask = np.array(mask)

    # Отрисовываем полигоны
    for tag, group in tags_dict.items():
        all_polygons = group['polygons']
        all_colors = group['colors']

        for polygon, color in zip(all_polygons, all_colors):
            mask = draw_filled_multipolygon(mask, polygon, bounds_coords, bounds_pxls, color)

    if show:
        cv2.imshow('mask', mask)
        cv2.waitKey(0)

    cv2.destroyAllWindows()
    return mask, mask_info
  
def def_aim_area(bounds):
    """
        Границы искомой области
    """
    n, s, e, w = bounds
    x_m = round((w+e)/2, 5)
    y_m = round((s+n)/2, 5)
    # lat, long, idx = y_m, x_m, 1
    lat, long = y_m, x_m

    d_lat = round((n-y_m), 5)
    d_long = round((e-x_m), 5)

    return lat, long, d_lat, d_long         
        
def defbounds(bounds, zoom):    
    """
    Определение границ сформированного из тайлов изображения

    Args:
        bounds (_type_): _description_
        zoom (_type_): _description_

    Returns:
        _type_: _description_
    """
    n, s, e, w = bounds
    
    tile_x1, tile_y1 = wgs_to_tile(w, n, zoom)
    tile_x2, tile_y2 = wgs_to_tile(e, s, zoom)

    latlt, lonlt = tile_to_lat_lon(tile_x1, tile_y1, zoom) # N, W
    latrb, lonrb = tile_to_lat_lon(tile_x2+1, tile_y2+1, zoom) # S, E
    print(f"Left-Top Latitude:\t {latlt}, \tLongitude: {lonlt}")
    print(f"Right-Bottom Latitude:\t {latrb}, \tLongitude: {lonrb}")
    bounds_coords = (lonlt, latlt, lonrb, latrb)
    return bounds_coords

def geojson_sort(gj, criterion=custom_sort_key):
    """Сортировка аннотаций для корректной отрисовки

    Args:
        gj (_type_): _description_
        criterion (_type_, optional): _description_. Defaults to custom_sort_key.

    Returns:
        _type_: _description_
    """
    features = []
    for obj in gj['features']:
        if obj['geometry']['type'] != 'Point':
            features.append(obj) 
    features = sorted(features, key=criterion) ##############
    return features

def create_background(shape, color):
    """_summary_

    Args:
        shape (_type_): (y, x) - shape of image
        color (_type_): (R,G,B) from 0 to 255

    Returns:
        _type_: _description_
    """
    # Задайте размеры изображения (x, y)
    y, x = shape
    # Создайте изображение
    image = Image.new('RGB', (x, y), color)

    return np.asarray(image)[:,:,::-1]

# ++++++++++++++++++++++++++++++++++++
def create_mapmask(
    geo_json, 
    save_folder, 
    name="VoidPlaceName",
    zoom=18,
    server="Google_wt_labels", 
    style = 's',
    background_color = (128,128,0)
    ):
    # server="Google_wt_labels"
    # style = 's' # m - map, s - satellite, y - satellite with labels
    
    # Чтение аннотаций
    with open(geo_json) as f:
        gj = geojson.load(f)

    # Сортировка аннотаций для корректной отрисовки
    features = geojson_sort(gj, criterion=custom_sort_key)

    download_path = f'{save_folder}/download'
    os.makedirs(download_path, exist_ok=True) 
    
    # Границы искомой области
    lat, long, d_lat, d_long = def_aim_area(bounds)
    # path = f'{download_path}/{name}_{idx}_{lat}_{long}_{D_LAT}_{D_LONG}_{STYLE}_{zoom}.bmp'
    # Загрузка и сшивка тайлов Google Maps
    path = f'{download_path}/{name}_{lat}_{long}_{d_lat}_{d_long}_{style}_{zoom}.bmp'
    nums = [lat + d_lat, long - d_long, lat - d_lat, long + d_long]
    top, left, bottom, right = nums
    if not os.path.isfile(path):
        download_map(left, top, right, bottom, zoom, path, style, server)

    # Определение границ сформированного из тайлов изображения
    bounds_coords = defbounds(bounds, zoom)

    img = cv2.imread(path)
    y,x,_ = img.shape  
    bounds_pxls = (0,0,x,y)

    img = create_background(img.shape[:2], background_color)
        
    # Преобразование мировых координат объекта в пиксели
    x0, y0, x1, y1 = world2pixels(bounds, bounds_coords, bounds_pxls)

    # Подготовка директорий
    mask_path = f'{save_folder}/masks'
    source_path = f'{save_folder}/sources'
    json_path = f'{save_folder}/jsons'
    for dirpath in [mask_path, source_path, json_path]:
        if not os.path.exists(dirpath):
            os.makedirs(dirpath)

    # Вычисление и отрисовка маски
    mask, mask_info = draw_masks(img, features, bounds_coords, bounds_pxls, zoom) 
    with open(f'{json_path}/{name}.json', 'w') as f:
        json.dump(mask_info, f)
    aim_area = mask[y0:y1, x0:x1]
    cv2.imwrite(f'{mask_path}/{name}.bmp', aim_area)

    # Сохранение целевой области с исходного изображения
    img = cv2.imread(path)
    img = img[y0:y1, x0:x1]
    cv2.imwrite(f'{source_path}/{name}.bmp', img)
    
# +++++++++++++++++++++
# Путь до информации об области
geo_json = 'data/Paris_running_road/Paris_running_road.geojson'
# Директория сохранения
save_folder = save_folder
# Имя области
name="Paris_running_road"

# Уровень зума (0-21)
zoom=18

# create_mapmask(geo_json, save_folder, name, zoom)

NameError: name 'save_folder' is not defined

In [3]:
# Чтение аннотаций
with open(geo_json) as f:
    gj = geojson.load(f)

# Сортировка аннотаций для корректной отрисовки
features = geojson_sort(gj, criterion=custom_sort_key)

# Определение границ сформированного из тайлов изображения
bounds_coords = defbounds(bounds, zoom)

path = 'data/Paris_running_road/download/Paris_running_road_48.94475_2.33242_0.00075_0.00173_s_18.bmp'
img = cv2.imread(path)
y,x,_ = img.shape  
bounds_pxls = (0,0,x,y)

opacity = 0.5
show = True
mask_info = []
tags_dict = {}

# Группируем объекты по тегам
for map_obj in features:
    mask_type = map_obj['geometry']['type']
    if mask_type == 'Point':
        continue

    coordinates = map_obj['geometry']['coordinates']
    
    if mask_type == 'Polygon':
        coordinates = [[coordinates[0]] + coordinates[1:]]  # Внешний контур + внутренние отверстия
    elif mask_type == 'MultiPolygon':
        coordinates = [[[poly[0]] + poly[1:]] for poly in coordinates]  # Для каждого полигона в MultiPolygon

    color = tuple(reversed(map_obj['properties']['color']))  # Преобразуем в BGR
    alpha = int(opacity * 255)

    tag = map_obj['properties']['tag']
    if tag not in tags_dict:
        tags_dict[tag] = {'polygons': [], 'colors': []}

    transformed_polygons = []
    for poly_set in coordinates:
        projected_polygons = []
        for poly in poly_set:
            if isinstance(poly[0][0], list):  # Если `poly` содержит вложенные списки, обрабатываем их отдельно
                processed_rings = [geocoordinates_to_pixels(ring, bounds_coords, bounds_pxls) for ring in poly]
                projected_polygons.append(processed_rings)
            else:
                print(poly)
                print()
                projected_polygons.append(geocoordinates_to_pixels(poly, bounds_coords, bounds_pxls))

        transformed_polygons.append(projected_polygons)

    tags_dict[tag]['polygons'].append(transformed_polygons)
    tags_dict[tag]['colors'].append((*color, alpha))

    mask_info.append({'tag': map_obj['properties']['tag'], 'coords': coordinates, 'color': color})

mask = np.array(mask)

# Отрисовываем полигоны
for tag, group in tags_dict.items():
    all_polygons = group['polygons']
    all_colors = group['colors']

    for polygon, color in zip(all_polygons, all_colors):
        mask = draw_filled_multipolygon(mask, polygon, bounds_coords, bounds_pxls, color)

if show:
    cv2.imshow('mask', mask)
    cv2.waitKey(0)

cv2.destroyAllWindows()

NameError: name 'geo_json' is not defined

In [34]:
def draw_filled_multipolygon(image, mp, bounds_coords, bounds_pxls):
    """
    Заполняет пространство между полигонами в мультиполигоне, сохраняя фон внутри фигур.
    """
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    tag, outer_contour, inner_contours, color = mp
    
    cv2.fillPoly(mask, [outer_contour], 255)  # Заполняем внешний контур
    for hole in inner_contours:
        hole = np.array(hole)
        try:
            cv2.fillPoly(mask, [hole], 0)  # Вырезаем внутренние отверстия
        except Exception as ex:
            print(ex)
            print(hole)
            print('{{{{}}}}')

    # Закрашиваем маску цветом
    color_bgr = color[:3]  # Берем только 3 канала (BGR)
    color_layer = np.full_like(image, color_bgr, dtype=np.uint8)
    image[mask == 255] = color_layer[mask == 255]
    
    return image

In [ ]:
north, south, east, west =  48.945501, 48.943991, 2.334149, 2.330695 # Paris running road
north, south, east, west =  48.950000, 48.943961, 2.334906, 2.321087 # Paris
bounds = north, south, east, west
zoom = 18

geo_json = 'data/Paris_running_road/Paris_running_road.geojson'
geo_json = 'data/paris/paris.geojson'
# Чтение аннотаций
with open(geo_json) as f:
    gj = geojson.load(f)

# Сортировка аннотаций для корректной отрисовки
features = geojson_sort(gj, criterion=custom_sort_key)

# Определение границ сформированного из тайлов изображения
bounds_coords = defbounds(bounds, zoom)

path = 'data/Paris_running_road/download/Paris_running_road_48.94475_2.33242_0.00075_0.00173_s_18.bmp'
path = 'data/paris/download/paris_48.95028_2.30675_0.00302_0.00691_s_18.bmp'
img = cv2.imread(path)
y,x,_ = img.shape  
bounds_pxls = (0,0,x,y)

opacity = 1
show = True
mask_info = []
tags_dict = {}
multipoligons = []

line_thickness = 3
mask = img.copy()
# def draw_mask =====================================
for map_obj in tqdm(features):
    tag = map_obj['properties']['tag']
    if tag not in tags_dict:
        tags_dict[tag] = {'polygons': [], 'polylines': [], 'line_thicknesses': [], 'colors': []}
    
    color = tuple(reversed(map_obj['properties']['color']))  # Преобразуем в BGR
    alpha = int(opacity * 255)  # Преобразуем прозрачность    
    
    mask_type = map_obj['geometry']['type']
    if mask_type == 'Point':
        continue

    coordinates = map_obj['geometry']['coordinates']

    if mask_type in ['Polygon', 'MultiPolygon']:
        if len(coordinates) > 1:
            # Обрабатываем внешний контур
            # Обработка кривых массивов (вместо одинарной вложенности может появиться двойная)
            outer_coords =  coordinates[0]
            if len(outer_coords) == 1:
                outer_coords = outer_coords[0]
            if len(outer_coords) == 1:
                continue

            outer_contour = np.array(
                [
                    geocoordinates_to_pixels(coord, bounds_coords, bounds_pxls) 
                        for coord in outer_coords
                ], dtype=np.int32
            )
            # Обрабатываем внутренние контуры
            inner_contours = []
            inner_coords = coordinates[1:]
            if len(inner_coords[0]) == 1:
                inner_coords = inner_coords[0]
            if len(inner_coords) == 1:
                continue
            for p in inner_coords:
                print(len(p), p)
                pp = [geocoordinates_to_pixels(coord, bounds_coords, bounds_pxls) for coord in p]
                inner_contours.append(pp)
            multipoligons.append([tag, outer_contour, inner_contours, color])

        else:
            poly_coords = coordinates[0]
            pixels = np.array([geocoordinates_to_pixels(coord, bounds_coords, bounds_pxls) 
                                for coord in poly_coords], dtype=np.int32)
            tags_dict[tag]['polygons'].append(pixels)
            tags_dict[tag]['colors'].append((*color, alpha))  # Добавляем альфа-канал

    elif mask_type == 'LineString':
        pixels = np.array([geocoordinates_to_pixels(coord, bounds_coords, bounds_pxls) 
                            for coord in coordinates], dtype=np.int32)
        tags_dict[tag]['polylines'].append(pixels)
        tags_dict[tag]['line_thicknesses'].append(line_thickness)
        tags_dict[tag]['colors'].append(color)
    else:
        print('other')
        
mask = np.array(mask)
# Отрисовываем полигоны и линии в порядке тегов
for tag, group in tags_dict.items():
    # Сначала отрисовываем полигоны для этого тега
    for polygon, color in zip(group['polygons'], group['colors']):
        mask = cv2.fillPoly(mask, [polygon], color)
    
    # Затем отрисовываем линии для этого тега
    for polyline, color, thickness in zip(group['polylines'], group['colors'], group['line_thicknesses']):
        mask = cv2.polylines(mask, [polyline], isClosed=False, color=color, thickness=thickness)
        
for mp in multipoligons:
        draw_filled_multipolygon(mask, mp, bounds_coords, bounds_pxls)

if show:
    cv2.imshow('mask', mask)
    cv2.waitKey(0)

cv2.destroyAllWindows()

print('+'*20)
    
    

Left-Top Latitude:	 48.950464623408834, 	Longitude: 2.32086181640625
Right-Bottom Latitude:	 48.94324925623428, 	Longitude: 2.335968017578125


 89%|████████▉ | 961/1077 [00:00<00:00, 112326.56it/s]

24 [[2.329044, 48.918374], [2.329053, 48.91838], [2.329242, 48.918507], [2.329244, 48.918516], [2.329244, 48.918528], [2.329238, 48.918537], [2.329229, 48.918546], [2.329213, 48.918551], [2.329193, 48.918555], [2.329176, 48.918555], [2.329162, 48.918552], [2.32915, 48.918547], [2.328977, 48.918434], [2.328968, 48.918427], [2.328959, 48.918416], [2.328957, 48.918407], [2.328957, 48.918399], [2.328962, 48.91839], [2.328969, 48.918385], [2.328979, 48.918379], [2.328994, 48.918372], [2.329012, 48.918369], [2.329029, 48.91837], [2.329044, 48.918374]]
27 [[2.328733, 48.918602], [2.32887, 48.918688], [2.328879, 48.918698], [2.32888, 48.918709], [2.328877, 48.918718], [2.328867, 48.918728], [2.328856, 48.918733], [2.328844, 48.918736], [2.328834, 48.918737], [2.328825, 48.918737], [2.328811, 48.918735], [2.328801, 48.918731], [2.32866, 48.918644], [2.328654, 48.918638], [2.328651, 48.91863], [2.328651, 48.918623], [2.328655, 48.918613], [2.32866, 48.918607], [2.328664, 48.918604], [2.32867, 48

TypeError: unsupported operand type(s) for -: 'list' and 'float'

In [40]:
coord

NameError: name 'coord' is not defined

In [ ]:
for f in features:
    if f['properties']['tag'] == 'sport':
        # print(f)
        g = f['geometry']['coordinates']

# g[1]

[[2.331827, 48.945205],
 [2.331698, 48.945225],
 [2.331589, 48.945217],
 [2.33151, 48.945205],
 [2.331432, 48.945177],
 [2.331358, 48.945135],
 [2.331296, 48.94509],
 [2.331238, 48.945042],
 [2.3312, 48.944984],
 [2.33117, 48.944925],
 [2.33115, 48.944862],
 [2.331151, 48.944792],
 [2.331166, 48.944731],
 [2.331207, 48.944668],
 [2.33126, 48.944617],
 [2.33133, 48.94458],
 [2.331408, 48.944544],
 [2.332324, 48.944277],
 [2.332426, 48.944248],
 [2.33251, 48.944227],
 [2.33261, 48.944222],
 [2.332697, 48.944228],
 [2.33279, 48.944246],
 [2.332876, 48.944274],
 [2.33295, 48.944312],
 [2.33301, 48.944361],
 [2.333062, 48.944416],
 [2.333104, 48.94447],
 [2.33313, 48.944531],
 [2.333139, 48.944592],
 [2.33314, 48.944656],
 [2.333121, 48.94472],
 [2.333084, 48.944782],
 [2.333023, 48.944832],
 [2.332983, 48.94486],
 [2.332933, 48.944882],
 [2.332865, 48.944904],
 [2.331827, 48.945205]]

In [ ]:
# Other

In [70]:
import cv2
import numpy as np
import random

def generate_multipolygon(num_polygons, num_layers, shape_type="circle", size_decrement=30, sides=6, img_size=(500, 500)):
    """
    Генерирует мультиполигон с несколькими независимыми полигонами, каждый из которых может содержать вложенные фигуры.

    :param num_polygons: Количество отдельных полигонов
    :param num_layers: Максимальное количество вложенных фигур внутри одного полигона
    :param shape_type: Тип фигуры ('circle', 'rectangle', 'triangle', 'polygon')
    :param size_decrement: Насколько уменьшается каждая вложенная фигура
    :param sides: Количество сторон для многоугольников
    :param img_size: Размер изображения (H, W)
    :return: Список мультиполигонов (каждый - список пар внешнего и внутреннего полигона)
    """
    height, width = img_size
    all_multipolygons = []

    for _ in range(num_polygons):
        center = (random.randint(100, width - 100), random.randint(100, height - 100))
        num_inner_layers = random.randint(1, num_layers)  # Случайное количество вложенных фигур

        polygon_set = []
        prev_outer = None

        for i in range(num_inner_layers):
            size = (num_inner_layers - i) * size_decrement  # Размер текущего слоя

            if shape_type == "circle":
                outer_points = np.array([
                    [int(center[0] + size * np.cos(angle)), int(center[1] + size * np.sin(angle))]
                    for angle in np.linspace(0, 2 * np.pi, 100)
                ], dtype=np.int32)

            elif shape_type == "rectangle":
                half_size = size // 2
                outer_points = np.array([
                    [center[0] - half_size, center[1] - half_size],
                    [center[0] + half_size, center[1] - half_size],
                    [center[0] + half_size, center[1] + half_size],
                    [center[0] - half_size, center[1] + half_size]
                ], dtype=np.int32)

            elif shape_type == "triangle":
                outer_points = np.array([
                    [center[0], center[1] - size],
                    [center[0] - size, center[1] + size],
                    [center[0] + size, center[1] + size]
                ], dtype=np.int32)

            elif shape_type == "polygon":
                outer_points = np.array([
                    [int(center[0] + size * np.cos(2 * np.pi * j / sides)),
                     int(center[1] + size * np.sin(2 * np.pi * j / sides))]
                    for j in range(sides)
                ], dtype=np.int32)

            if prev_outer is not None:
                polygon_set.append([prev_outer, outer_points])

            prev_outer = outer_points  # Запоминаем внешний контур как предыдущий

        all_multipolygons.append(polygon_set)

    return all_multipolygons

def draw_filled_multipolygon(image, multipolygon, color):
    """
    Заполняет пространство между полигонами в мультиполигоне, сохраняя фон внутри фигур.

    :param image: Исходное изображение
    :param multipolygon: Список мультиполигонов
    :param color: Цвет заливки (BGR)
    """
    mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

    for polygon_set in multipolygon:
        for outer_points, inner_points in polygon_set:
            cv2.fillPoly(mask, [outer_points], 255)
            cv2.fillPoly(mask, [inner_points], 0)

    colored_layer = np.full_like(image, color, dtype=np.uint8)
    image[mask == 255] = colored_layer[mask == 255]

# === Параметры ===
height, width = 500, 500
random_image = np.random.randint(0, 256, (height, width, 3), dtype=np.uint8)

num_polygons = 3  # Количество независимых полигонов
num_layers = 2  # Максимальное количество вложенных уровней в каждом полигоне
shape_type = "polygon"  # "circle", "rectangle", "triangle", "polygon"
color = (0, 255, 0)
sides = 5  # Для многоугольников

# === Генерация и отрисовка ===
multipolygon = generate_multipolygon(num_polygons, num_layers, shape_type, size_decrement=40, sides=sides, img_size=(height, width))
draw_filled_multipolygon(random_image, multipolygon, color)

# Показать изображение
cv2.imshow('Complex Multipolygon', random_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [64]:
random_image = np.random.randint(0, 256, (height, width, 3), dtype=np.uint8)
draw_filled_multipolygon(random_image, multipolygon[0], color)

# Показать изображение
cv2.imshow('Complex Multipolygon', random_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

ValueError: too many values to unpack (expected 2)

In [68]:
multipolygon[0]

[[array([[387, 319],
         [304, 433],
         [169, 389],
         [169, 248],
         [304, 204]], dtype=int32),
  array([[347, 319],
         [291, 395],
         [202, 366],
         [202, 271],
         [291, 242]], dtype=int32)],
 [array([[347, 319],
         [291, 395],
         [202, 366],
         [202, 271],
         [291, 242]], dtype=int32),
  array([[307, 319],
         [279, 357],
         [234, 342],
         [234, 295],
         [279, 280]], dtype=int32)]]